# Generating melody *and* harmony using LSTMs

*** NOTE: This does the same thing as the [`melody_harmony_tag_lstm.ipynb`](melody_harmony_tag_lstm.ipynb) notebook, except that here we use the `jazzaiexperiments` module instead of defining everything inside the notebook itself. ***

Again, using data from the Weimar Jazz Database. This time we make use of the beat onset and chord information from the database in addition to the converted MIDI files.

Audio examples live at https://soundcloud.com/usdivad/sets/jazz-ai-experiments under "melody + harmony tag"; links to individual tracks are at the bottom of this notebook.

## Approach #1: Add underlying harmony directly to note events

Import everything we need:

In [1]:
import os
import jazzaiexperiments

Using TensorFlow backend.


### Putting it all together

Train the model:

In [2]:
tune_name, time_multiplier, num_notes_to_generate = ("JohnColtrane_GiantSteps-1_FINAL", 2.5, 200)
beats_filepath = jazzaiexperiments.db.construct_filepath(tune_name, "beats")
melody_filepath = jazzaiexperiments.db.construct_filepath(tune_name, "melody")

In [3]:
trained = jazzaiexperiments.lstm.train_on_midi_input(tune_name=tune_name,
                                                     mode="single_melody_harmony",
                                                     midi_data_dir="../data/midi/quantized/",
                                                     db_beats_filepath=beats_filepath,
                                                     db_melody_filepath=melody_filepath,
                                                     seq_length=10,
                                                     num_epochs=1)
model, note_events, input_filepath = trained

Created 1127 note events from ../data/midi/quantized/JohnColtrane_GiantSteps-1_FINAL.mid using mode single_melody_harmony
Formatted note data (1117 seqs of length 10, 601 unique notes)
Created model
No weights loaded (to load weights, specify a `weights_filepath`)
Epoch 1/1
1117/1117 [==============================] - 5s - loss: 6.3609     
Trained model over 1 epochs


Load weights ay:

In [4]:
jazzaiexperiments.lstm.load_model_weights(model, "../data/models/weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_99_0.7281.hdf5")

Use the model to generate some notes:

In [5]:
notes_out, output_filepath = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
                                                                         mode="single_melody_harmony",
                                                                         num_notes_to_generate=num_notes_to_generate,
                                                                         random_seed=False,
                                                                         add_seed_to_output=True,
                                                                         tune_name=tune_name,
                                                                         time_multiplier=time_multiplier,
                                                                         midi_source_filepath=input_filepath,
                                                                         data_dir="../data/output")
notes_out[:20]

Constructed input sequence: [154, 254, 395, 529, 475, 395, 313, 212, 106, 186]
Generated 200 notes
Wrote to MIDI file at ../data/output/out_JohnColtrane_GiantSteps-1_FINAL_20170705214855015292.mid


[(61, 110, 0, 96, 'NC'),
 (64, 110, 0, 96, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (71, 120, 0, 96, 'NC'),
 (70, 110, 0, 288, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (66, 110, 0, 96, 'Cbj7'),
 (63, 110, 0, 96, 'Cbj7'),
 (59, 100, 0, 240, 'Cbj7'),
 (62, 110, 0, 96, 'D7'),
 (64, 110, 0, 48, 'D7'),
 (66, 100, 0, 96, 'D7'),
 (69, 110, 0, 96, 'D7'),
 (67, 110, 0, 96, 'Gj7'),
 (62, 100, 0, 96, 'Gj7'),
 (59, 100, 0, 96, 'Gj7'),
 (59, 100, 0, 96, 'Bb7'),
 (56, 110, 0, 96, 'Bb7'),
 (55, 100, 0, 96, 'Bb7'),
 (53, 100, 0, 96, 'Bb7')]

In [6]:
# Trying to do MIDI -> mp3 conversion directly in notebook
# %%bash -s "$output_filepath"
# source ~/.bash_profile
# miditomp3 $1

In [7]:
# Play the MIDI file here
# music21.converter.parse("../data/output/out_ColemanHawkins_BodyAndSoul_FINAL_20170703163206182499.mid").show("midi")

Load multiple weights and generate output for each of them (so we can compare):

In [8]:
weights_filepaths = []

if tune_name == "ColemanHawkins_BodyAndSoul_FINAL":  # Coleman Hawkins - Body and Soul
    weights_filepaths = ["weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_00_6.3755.hdf5",
                         "weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_09_5.4847.hdf5",
                         "weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_49_1.5432.hdf5",
                         "weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_99_0.4374.hdf5",
                         "weights_ColemanHawkins_BodyAndSoul_FINAL_20170703165925088780_146_0.2167.hdf5"]

elif tune_name == "JohnColtrane_GiantSteps_FINAL":  # John Coltrane - Giant Steps
    weights_filepaths = ["weights_JohnColtrane_GiantSteps-1_FINAL_20170703180209718872_00_6.3668.hdf5",
                         "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_09_5.7838.hdf5",
                         "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_49_2.0546.hdf5",
                         "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_99_0.7281.hdf5"]

elif tune_name == "CharlieParker_DonnaLee_FINAL":  # Charlie Parker - Donna Lee
    weights_filepaths = ["weights_CharlieParker_DonnaLee_FINAL_20170703185151689429_00_5.7642.hdf5",
                         "weights_CharlieParker_DonnaLee_FINAL_20170703185151689429_09_5.1143.hdf5",
                         "weights_CharlieParker_DonnaLee_FINAL_20170703185151689429_49_1.6850.hdf5",
                         "weights_CharlieParker_DonnaLee_FINAL_20170703185151689429_91_0.8709.hdf5"]
else:
    print("No weights found for tune {}".format(tune_name))

weights_filepaths = [os.path.join("../data/models/", path) for path in weights_filepaths]

for weights_filepath in weights_filepaths:
    jazzaiexperiments.lstm.load_weights(model, weights_filepath)
    print("Loaded weights from {}".format(weights_filepath))

    output = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
                                                         mode="single_melody_harmony",
                                                         num_notes_to_generate=num_notes_to_generate,
                                                         random_seed=False,
                                                         add_seed_to_output=True,
                                                         tune_name=tune_name,
                                                         time_multiplier=time_multiplier,
                                                         midi_source_filepath=input_filepath,
                                                         data_dir="../data/output")
print("Done writing")

No weights found for tune JohnColtrane_GiantSteps-1_FINAL
Done writing


Render the original input notes with chords:

In [9]:
jazzaiexperiments.midi.write_file(note_events, os.path.join("../data/output/", "{}_original.mid".format(tune_name)),
                                  mode="single_melody_harmony",
                                  time_multiplier=time_multiplier,
                                  midi_source_filepath=input_filepath)
# print("Wrote to MIDI file at {}".format(output_filepath))

'../data/output/JohnColtrane_GiantSteps-1_FINAL_original.mid'

(

Run this whenever we want to train the model even further (but don't forget to update the `weights_filepath`):

In [10]:
# trained = jazzaiexperiments.lstm.train_on_midi_input(tune_name=tune_name,
#                                                      mode="single_melody_harmony",
#                                                      midi_data_dir="../data/midi/quantized/",
#                                                      db_beats_filepath=beats_filepath,
#                                                      db_melody_filepath=melody_filepath,
#                                                      weights_filepath=None,
#                                                      seq_length=10,
#                                                      num_epochs=100)
# model, note_events, input_filepath = trained

)

### Audio examples

Method:
- LSTM with 256 units in input layer, 1 hidden layer with 256 units, sequence length of 10 notes
- Each MIDI note is "tagged" with the chord that it was originally played over
- Seeded with the first 10 notes of the original transcription (which are prepended to the output)

Using Coleman Hawkins - Body and Soul:
- 10 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-melody-harmony-tag-coleman-hawkins-body-and-soul-10-epochs
- 50 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-melody-harmony-tag-coleman-hawkins-body-and-soul-50-epochs
- 100 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-melody-harmony-tag-coleman-hawkins-body-and-soul-100-epochs

Using John Coltrane - Giant Steps:
- 50 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-melody-harmony-tag-john-coltrane-giant-steps-50-epochs
- 100 epochs: https://soundcloud.com/usdivad/jazz-ai-experiments-lstm-melody-harmony-tag-john-coltrane-giant-steps-100-epochs



*TODO: Discuss the results*